In [1]:
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
from pathlib import Path

In [2]:
def load_sample(sample_path):
    with open(sample_path, 'r') as f:
        samples = json.load(f)
    return samples

In [3]:
def prepare_cropped_image(sample_entry, target_size=(128, 128)):
    rgb_path = Path(sample_entry['rgb_path'].replace('\\', '/'))
    img = cv2.imread(str(rgb_path))
    if img is None:
        raise FileNotFoundError(f"Image not found: {rgb_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Crop using bbox_obj
    x, y, w, h = sample_entry['bbox_obj']
    cropped_img = img_rgb[y:y+h, x:x+w]

    # Resize cropped image
    resized_img = cv2.resize(cropped_img, target_size)

    # Adjust keypoints relative to crop
    keypoints_2D = np.array(sample_entry['keypoints_2D'])
    keypoints_2D_cropped = keypoints_2D - np.array([x, y])

    # Scale keypoints to new size
    scale_x = target_size[0] / w
    scale_y = target_size[1] / h
    keypoints_2D_resized = keypoints_2D_cropped * np.array([scale_x, scale_y])

    return resized_img, keypoints_2D_resized, sample_entry['image_id']

In [4]:
def visualize_multiple_samples(samples):
    fig, axes = plt.subplots(1, len(samples), figsize=(5 * len(samples), 5))

    if len(samples) == 1:
        axes = [axes]

    for ax, sample_entry in zip(axes, samples):
        resized_img, keypoints_2D_resized, image_id = prepare_cropped_image(sample_entry)
        ax.imshow(resized_img)
        ax.axis('off')

        for idx, (x_kp, y_kp) in enumerate(keypoints_2D_resized):
            ax.scatter(x_kp, y_kp, c='red', s=40)
            ax.text(x_kp + 2, y_kp - 2, str(idx), color='yellow', fontsize=8)

        ax.set_title(f"Image ID: {image_id}")

    plt.tight_layout()
    plt.show()

In [5]:
obj_id = 1  # Change as needed
sample_path = Path(f"../data/annotations/train/{obj_id:06d}.json")
samples = load_sample(sample_path)

# Randomly select 3 samples
selected_samples = random.sample(samples, 3)

visualize_multiple_samples(selected_samples)


FileNotFoundError: [Errno 2] No such file or directory: '../data/annotations/train/000001.json'